In [19]:
import pandas as pd
import scraper
import json
import numpy as np

In [20]:
#https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json

In [23]:

def monthWindow(df):    
    daysDiff = df['diffDate']
    months = 0
    while daysDiff >= 30:
        daysDiff -=30
        months +=1
        
    return months


def unpactUnitsJson(index, jsonDataframe):
    key = list(jsonDataframe['units'].iloc[index].keys())[0]
    valuesInTable = pd.DataFrame(jsonDataframe['units'].iloc[index][key])
        
    if 'start' in valuesInTable.columns:
        valuesInTable['startFormat'] = pd.to_datetime(valuesInTable['start'])
    
    if 'end' in valuesInTable.columns:
        valuesInTable['endFormat'] = pd.to_datetime(valuesInTable['end'])
        
    if 'end' in valuesInTable.columns and 'start' in valuesInTable.columns:
        valuesInTable['diffDate'] =  (valuesInTable['endFormat'] - valuesInTable['startFormat']).dt.days
        valuesInTable['monthWindow'] = valuesInTable.apply(monthWindow, axis=1)
        
    valuesInTable['time'] = valuesInTable['endFormat'].dt.year.astype(str).str[2:] + valuesInTable['endFormat'].dt.month.map("{:02}".format)

    
    return valuesInTable


def get_cleaned_companyfacts(jsonDataframe):
    valuesDF = pd.DataFrame()
    for index, row in jsonDataframe.iterrows():
        unit = unpactUnitsJson(index, jsonDataframe)
        unit['finType'] = jsonDataframe.iloc[index]['finType']
        valuesDF = pd.concat([unit,valuesDF])
        
        
    mergedDF = jsonDataframe.merge(valuesDF, on='finType')
    mergedDF.drop(['units'], axis = 1, inplace=True)
    mergedDF.fillna("null", inplace = True)
    
    return mergedDF

In [36]:
#CIK0000320193
def get_companyfacts(cik):
    if "CIK" not in cik:
        cik = "CIK" + cik
    
    baseUrl = f'https://data.sec.gov/api/xbrl/companyfacts/{cik}.json'
    scr = scraper.APIconnector(baseUrl)
    jsonRequest = scr.get_request().json()
    r = jsonRequest['facts']['us-gaap']
    r = json.dumps(r)
    jsonDataframe = pd.read_json(r).T

    jsonDataframe.reset_index(inplace=True, names='finType')
    
    mergedDF = get_cleaned_companyfacts(jsonDataframe)
    
    return mergedDF

In [1]:
from scraper import get_companyfacts

In [2]:
    
mergedDF = get_companyfacts('CIK0000320193')

mergedDF

,finType,label,description,end,val,accn,fy,fp,form,filed,frame,endFormat,time,start,startFormat,diffDate,monthWindow
0,AccountsPayable,Accounts Payable (Deprecated 2009-01-31),Carrying value as of the balance sheet date of...,2008-09-27,5.520000e+09,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2008Q3I,2008-09-27,0809,null,null,null,null
1,AccountsPayable,Accounts Payable (Deprecated 2009-01-31),Carrying value as of the balance sheet date of...,2009-06-27,4.854000e+09,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I,2009-06-27,0906,null,null,null,null
2,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,2008-09-27,5.520000e+09,0001193125-09-214859,2009,FY,10-K,2009-10-27,null,2008-09-27,0809,null,null,null,null
3,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,2008-09-27,5.520000e+09,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,CY2008Q3I,2008-09-27,0809,null,null,null,null
4,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,2009-09-26,5.601000e+09,0001193125-09-214859,2009,FY,10-K,2009-10-27,null,2009-09-26,0909,null,null,null,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22738,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,2024-03-30,5.885300e+07,0000320193-24-000069,2024,Q2,10-Q,2024-05-03,CY2024Q1,2024-03-30,2403,2023-12-31,2023-12-31 00:00:00,90.0,3.0
22739,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,2024-06-29,6.212800e+07,0000320193-24-000081,2024,Q3,10-Q,2024-08-02,null,2024-06-29,2406,2023-10-01,2023-10-01 00:00:00,272.0,9.0
22740,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,2024-06-29,6.065400e+07,0000320193-24-000081,2024,Q3,10-Q,2024-08-02,CY2024Q2,2024-06-29,2406,2024-03-31,2024-03-31 00:00:00,90.0,3.0
22741,OtherAssetsMiscellaneousNoncurrent,"Other Assets, Miscellaneous, Noncurrent",Amount of other miscellaneous assets expected ...,2022-09-24,3.905300e+10,0000320193-23-000106,2023,FY,10-K,2023-11-03,CY2022Q3I,2022-09-24,2209,null,null,null,null


In [22]:
index = 4

valuesInTable = pd.DataFrame(jsonDataframe['units'].iloc[index]['USD'])
valuesInTable

,end,val,accn,fy,fp,form,filed,frame
0,2017-09-30,257000000,0000320193-18-000007,2018,Q1,10-Q,2018-02-02,NaN
1,2017-09-30,257000000,0000320193-18-000070,2018,Q2,10-Q,2018-05-02,NaN
2,2017-09-30,257000000,0000320193-18-000100,2018,Q3,10-Q,2018-08-01,NaN
3,2017-09-30,257000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q3I
4,2017-12-30,34913000000,0000320193-18-000007,2018,Q1,10-Q,2018-02-02,CY2017Q4I
5,2018-03-31,34913000000,0000320193-18-000070,2018,Q2,10-Q,2018-05-02,CY2018Q1I
6,2018-06-30,34029000000,0000320193-18-000100,2018,Q3,10-Q,2018-08-01,CY2018Q2I
7,2018-09-29,33589000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,NaN
8,2018-09-29,33589000000,0000320193-19-000010,2019,Q1,10-Q,2019-01-30,NaN
9,2018-09-29,33589000000,0000320193-19-000066,2019,Q2,10-Q,2019-05-01,NaN


In [35]:
mergedDF

,finType,label,description,end,val,accn,fy,fp,form,filed,frame,endFormat,time,start,startFormat,diffDate,monthWindow
0,AccountsPayable,Accounts Payable (Deprecated 2009-01-31),Carrying value as of the balance sheet date of...,2008-09-27,5.520000e+09,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2008Q3I,2008-09-27,0809,null,null,null,null
1,AccountsPayable,Accounts Payable (Deprecated 2009-01-31),Carrying value as of the balance sheet date of...,2009-06-27,4.854000e+09,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I,2009-06-27,0906,null,null,null,null
2,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,2008-09-27,5.520000e+09,0001193125-09-214859,2009,FY,10-K,2009-10-27,null,2008-09-27,0809,null,null,null,null
3,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,2008-09-27,5.520000e+09,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,CY2008Q3I,2008-09-27,0809,null,null,null,null
4,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,2009-09-26,5.601000e+09,0001193125-09-214859,2009,FY,10-K,2009-10-27,null,2009-09-26,0909,null,null,null,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22738,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,2024-03-30,5.885300e+07,0000320193-24-000069,2024,Q2,10-Q,2024-05-03,CY2024Q1,2024-03-30,2403,2023-12-31,2023-12-31 00:00:00,90.0,3.0
22739,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,2024-06-29,6.212800e+07,0000320193-24-000081,2024,Q3,10-Q,2024-08-02,null,2024-06-29,2406,2023-10-01,2023-10-01 00:00:00,272.0,9.0
22740,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,2024-06-29,6.065400e+07,0000320193-24-000081,2024,Q3,10-Q,2024-08-02,CY2024Q2,2024-06-29,2406,2024-03-31,2024-03-31 00:00:00,90.0,3.0
22741,OtherAssetsMiscellaneousNoncurrent,"Other Assets, Miscellaneous, Noncurrent",Amount of other miscellaneous assets expected ...,2022-09-24,3.905300e+10,0000320193-23-000106,2023,FY,10-K,2023-11-03,CY2022Q3I,2022-09-24,2209,null,null,null,null
